In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# ===== Load and prepare data =====
def load_and_prepare_data(filepath):
    data = pd.read_csv(filepath)
    data.drop(columns=["Neighborhood", "Home"], inplace=True)
    data["Brick"] = data["Brick"].map({"No": 0, "Yes": 1})
    
    X = data.drop(columns="Price").values.astype(float)
    y = data["Price"].values.reshape(-1, 1).astype(float)
    
    return X, y, data.columns.tolist()

# ===== Scale data =====
def scale_data(X_train, X_test, y_train):
    # Scale features
    X_mean = X_train.mean(axis=0)
    X_std = X_train.std(axis=0)
    X_std[X_std == 0] = 1
    X_train_s = (X_train - X_mean) / X_std
    X_test_s = (X_test - X_mean) / X_std
    
    # Scale target
    y_mean = y_train.mean()
    y_std = y_train.std()
    y_train_s = (y_train - y_mean) / y_std
    
    return X_train_s, X_test_s, y_train_s, X_mean, X_std, y_mean, y_std

# ===== Neural Network with ReLU =====
class NeuralNetwork:
    def __init__(self, input_size, hidden1=32, hidden2=16, output_size=1, learning_rate=0.01):
        np.random.seed(42)
        
        # Initialize weights with Xavier initialization
        self.w1 = np.random.randn(input_size, hidden1) * np.sqrt(2.0 / (input_size + hidden1))
        self.b1 = np.zeros((1, hidden1))
        self.w2 = np.random.randn(hidden1, hidden2) * np.sqrt(2.0 / (hidden1 + hidden2))
        self.b2 = np.zeros((1, hidden2))
        self.w3 = np.random.randn(hidden2, output_size) * np.sqrt(2.0 / (hidden2 + output_size))
        self.b3 = np.zeros((1, output_size))
        
        self.lr = learning_rate
        self.best_weights = None
        self.best_loss = float('inf')
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def relu_deriv(self, x):
        return np.where(x > 0, 1, 0)
    
    def forward(self, X):
        self.Z1 = X @ self.w1 + self.b1
        self.A1 = self.relu(self.Z1)
        self.Z2 = self.A1 @ self.w2 + self.b2
        self.A2 = self.relu(self.Z2)
        self.Z3 = self.A2 @ self.w3 + self.b3
        return self.Z3
    
    def backward(self, X, y, y_pred):
        m = X.shape[0]
        
        # Output layer gradients
        dZ3 = 2 * (y_pred - y) / m
        dW3 = self.A2.T @ dZ3
        db3 = np.sum(dZ3, axis=0, keepdims=True)
        
        # Hidden layer 2 gradients
        dA2 = dZ3 @ self.w3.T
        dZ2 = dA2 * self.relu_deriv(self.Z2)
        dW2 = self.A1.T @ dZ2
        db2 = np.sum(dZ2, axis=0, keepdims=True)
        
        # Hidden layer 1 gradients
        dA1 = dZ2 @ self.w2.T
        dZ1 = dA1 * self.relu_deriv(self.Z1)
        dW1 = X.T @ dZ1
        db1 = np.sum(dZ1, axis=0, keepdims=True)
        
        return dW1, db1, dW2, db2, dW3, db3
    
    def update_weights(self, dW1, db1, dW2, db2, dW3, db3):
        self.w1 -= self.lr * dW1
        self.b1 -= self.lr * db1
        self.w2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.w3 -= self.lr * dW3
        self.b3 -= self.lr * db3
    
    def train(self, X_train, y_train, epochs=5000, verbose=True):
        for epoch in range(epochs):
            # Forward pass
            y_pred = self.forward(X_train)
            
            # Compute loss
            loss = np.mean((y_pred - y_train)**2)
            
            # Save best weights
            if loss < self.best_loss:
                self.best_loss = loss
                self.best_weights = (
                    self.w1.copy(), self.b1.copy(),
                    self.w2.copy(), self.b2.copy(),
                    self.w3.copy(), self.b3.copy()
                )
            
            # Backward pass
            dW1, db1, dW2, db2, dW3, db3 = self.backward(X_train, y_train, y_pred)
            
            # Update weights
            self.update_weights(dW1, db1, dW2, db2, dW3, db3)
            
            # Learning rate decay
            if epoch % 1000 == 0 and epoch > 0:
                self.lr *= 0.9
            
            if verbose and epoch % 500 == 0:
                print(f"Epoch {epoch:4d} | Loss: {loss:.6f}")
        
        # Restore best weights
        if self.best_weights:
            self.w1, self.b1, self.w2, self.b2, self.w3, self.b3 = self.best_weights
    
    def predict(self, X, y_mean, y_std):
        predictions = self.forward(X)
        return predictions * y_std + y_mean

# ===== Main execution =====
def main():
    # Load data
    X, y, feature_names = load_and_prepare_data("house-prices.csv")
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Scale data
    X_train_s, X_test_s, y_train_s, X_mean, X_std, y_mean, y_std = scale_data(
        X_train, X_test, y_train
    )
    
    # Create and train neural network
    print("Training neural network...")
    nn = NeuralNetwork(
        input_size=X_train_s.shape[1],
        hidden1=32,
        hidden2=16,
        learning_rate=0.1
    )
    
    nn.train(X_train_s, y_train_s, epochs=90000, verbose=True)
    
    # Make predictions on test set
    y_pred = nn.predict(X_test_s, y_mean, y_std)
    
    # Calculate metrics
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"\n=== Model Performance on Test Set ===")
    print(f"MAE:  ${mae:,.2f}")
    print(f"RMSE: ${rmse:,.2f}")
    print(f"R²:   {r2:.4f}")
    
    # Predict your specific sample
    sample = np.array([[2250, 4, 3, 3, 1]])  # From Home #1
    sample_s = (sample - X_mean) / X_std
    sample_pred = nn.predict(sample_s, y_mean, y_std)
    
    print(f"\n=== Prediction for Sample House ===")
    print(f"Features: SqFt={sample[0,0]}, Bed={sample[0,1]}, Bath={sample[0,2]}, "
          f"Offers={sample[0,3]}, Brick={sample[0,4]}")
    print(f"Predicted price: ${sample_pred[0,0]:,.0f}")

    # Feature importance analysis
    print(f"\n=== Feature Importance ===")
    feature_importance = np.mean(np.abs(nn.w1), axis=1)
    for i, (feature, importance) in enumerate(zip(feature_names[:-1], feature_importance)):
        print(f"{feature:10} | Importance: {importance:.4f} | Correlation with Price: {np.corrcoef(X_train[:, i], y_train.flatten())[0, 1]:.4f}")
    
    return nn, X_mean, X_std, y_mean, y_std

if __name__ == "__main__":
    nn, X_mean, X_std, y_mean, y_std = main()
    
    # You can now use the trained model for new predictions:
    # new_house = np.array([[sqft, bedrooms, bathrooms, offers, brick]])
    # new_house_scaled = (new_house - X_mean) / X_std
    # predicted_price = nn.predict(new_house_scaled, y_mean, y_std)

Training neural network...
Epoch    0 | Loss: 1.238938
Epoch  500 | Loss: 0.096696
Epoch 1000 | Loss: 0.087517
Epoch 1500 | Loss: 0.079298
Epoch 2000 | Loss: 0.077099
Epoch 2500 | Loss: 0.067734
Epoch 3000 | Loss: 0.064467
Epoch 3500 | Loss: 0.059829
Epoch 4000 | Loss: 0.059309
Epoch 4500 | Loss: 0.053939
Epoch 5000 | Loss: 0.054247
Epoch 5500 | Loss: 0.047026
Epoch 6000 | Loss: 0.046568
Epoch 6500 | Loss: 0.041211
Epoch 7000 | Loss: 0.044144
Epoch 7500 | Loss: 0.038206
Epoch 8000 | Loss: 0.039139
Epoch 8500 | Loss: 0.036215
Epoch 9000 | Loss: 0.037867
Epoch 9500 | Loss: 0.033726
Epoch 10000 | Loss: 0.035133
Epoch 10500 | Loss: 0.031867
Epoch 11000 | Loss: 0.032637
Epoch 11500 | Loss: 0.029712
Epoch 12000 | Loss: 0.030072
Epoch 12500 | Loss: 0.027597
Epoch 13000 | Loss: 0.027874
Epoch 13500 | Loss: 0.025790
Epoch 14000 | Loss: 0.026055
Epoch 14500 | Loss: 0.024386
Epoch 15000 | Loss: 0.024029
Epoch 15500 | Loss: 0.022843
Epoch 16000 | Loss: 0.022127
Epoch 16500 | Loss: 0.021452
Epoch 1